In [ ]:
!pip install transformers accelerate sentencepiece datasets

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Use a small model first to understand quickly
model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Zero Shot Prompting

In [ ]:
question = "Where is Paris?"
prompt = f"Classify the question type: \"{question}\""

In [ ]:
prompt

'Classify the question type: "Where is Paris?"'

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=10)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Paris


In [ ]:
# Few Shot Prompting

In [ ]:
prompt = """Classify the question type:
Q: Where is New York?
A: LOC

Q: Who is the Prime Minister of India?
A: HUM

Q: How many grams is in kilograms?
A:
"""

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=10)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

gram


In [ ]:
# The output should be entity i.e. NUM, HUM, LOC. Clearly the zero-shot and few-shot prompting does not work.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("trec")
train_data = dataset["train"]
test_data = dataset["test"]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

trec.py:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

The repository for trec contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/trec.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
def convert_to_instruction_format(example):
    return {
        "input": f"Classify the question type: \"{example['text']}\"",
        "output": example["coarse_label"]
    }

train_data = train_data.map(convert_to_instruction_format)
test_data = test_data.map(convert_to_instruction_format)

Map:   0%|          | 0/5452 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Get label names
label_names = dataset['train'].features['coarse_label'].names
print(label_names)

['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM']


In [ ]:
coarse_label_map = {i: label for i, label in enumerate(label_names)}
print(coarse_label_map)

{0: 'ABBR', 1: 'ENTY', 2: 'DESC', 3: 'HUM', 4: 'LOC', 5: 'NUM'}


In [ ]:
train_data["input"][0]

'Classify the question type: "How did serfdom develop in and then leave Russia ?"'

In [ ]:
train_data["output"][0]

2

In [ ]:
def preprocess(example):
    input_text = str(example["input"])
    output_text = str(example["output"])

    model_inputs = tokenizer(
        input_text, max_length=128, truncation=True, padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            output_text, max_length=2, truncation=True, padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = train_data.map(preprocess, batched=False)
tokenized_test = test_data.map(preprocess, batched=False)

Map:   0%|          | 0/5452 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'coarse_label', 'fine_label', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5452
})

In [ ]:
tokenized_train["labels"][0]

[204, 1]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    eval_steps=400,  # You can tweak this number
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,

)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-23-28f6b3ff398d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: omkarwazulkar (omkarwazulkar1998) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,0.750200
200,0.176100
300,0.153300
400,0.134200
500,0.101600
600,0.097900
700,0.079000
800,0.057800
900,0.060900
1000,0.057400


TrainOutput(global_step=1364, training_loss=0.13491821953278482, metrics={'train_runtime': 293.7562, 'train_samples_per_second': 74.238, 'train_steps_per_second': 4.643, 'total_flos': 1013475031646208.0, 'train_loss': 0.13491821953278482, 'epoch': 4.0})

In [ ]:
model.save_pretrained("saved_model_")
tokenizer.save_pretrained("saved_model_")

('saved_model_/tokenizer_config.json',
 'saved_model_/special_tokens_map.json',
 'saved_model_/spiece.model',
 'saved_model_/added_tokens.json',
 'saved_model_/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/saved_model_")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/saved_model_")

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.07216385006904602, 'eval_runtime': 4.545, 'eval_samples_per_second': 110.012, 'eval_steps_per_second': 7.041, 'epoch': 4.0}


In [ ]:
train_data[3]

{'text': 'What fowl grabs the spotlight after the Chinese Year of the Monkey ?',
 'coarse_label': 1,
 'fine_label': 2,
 'input': 'Classify the question type: "What fowl grabs the spotlight after the Chinese Year of the Monkey ?"',
 'output': 1}

In [ ]:
inputs = tokenizer('Classify the question type: "What fowl grabs the spotlight after the Chinese Year of the Monkey ?"', return_tensors="pt").to(model.device)
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

1


In [ ]:
inputs = tokenizer('Classify the question type: "Who is the Prime Minister of India ?"', return_tensors="pt").to(model.device)
outputs = model.generate(**inputs)
predicted_label_index = int(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(coarse_label_map.get(predicted_label_index, "Unknown Label"))

HUM


In [ ]:
inputs = tokenizer('Classify the question type: "Where is Paris ?"', return_tensors="pt").to(model.device)
outputs = model.generate(**inputs)
predicted_label_index = int(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(coarse_label_map.get(predicted_label_index, "Unknown Label"))

LOC


In [ ]:
inputs = tokenizer('Classify the question type: "How many grams is in kilograms?"', return_tensors="pt").to(model.device)
outputs = model.generate(**inputs)
predicted_label_index = int(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(coarse_label_map.get(predicted_label_index, "Unknown Label"))

NUM


In [ ]:
prompt

'Classify the question type: \nQ: Where is New York?\nA: LOC\n\nQ: Who is the Prime Minister of India?\nA: HUM\n\nQ: How many grams is in kilograms?\nA:\n'

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs)
predicted_label_index = int(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(coarse_label_map.get(predicted_label_index, "Unknown Label"))

NUM


In [ ]:
from tqdm import tqdm
import torch

correct = 0
total = 0

model.eval()

for example in tqdm(test_data):
    input_text = example["input"]
    true_label = str(example["output"]).strip()  # convert label to string

    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=5)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Clean prediction (sometimes model outputs extra text)
    pred = pred.strip().replace("Label:", "").strip()

    if pred == true_label:
        correct += 1
    total += 1

accuracy = correct / total
print(f"\n📊 Final Accuracy: {accuracy * 100:.2f}% on {total} test examples")

100%|██████████| 500/500 [00:23<00:00, 21.65it/s]


📊 Final Accuracy: 96.20% on 500 test examples


In [ ]:
from fastapi import FastAPI, HTTPException # type: ignore
from pydantic import BaseModel # type: ignore
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch # type: ignore

# Load model and tokenizer
model_path = "./saved_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Use CPU or GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create FastAPI app
app = FastAPI(title="LLM Inference API")

# Request schema
class InputText(BaseModel):
    prompt: str

@app.post("/predict/")
def predict(input: InputText):
    try:
        # Tokenize and move to correct device
        inputs = tokenizer(
            input.prompt,
            return_tensors="pt",
            truncation=True,
            padding=True
        ).to(device)

        # Generate prediction
        outputs = model.generate(**inputs, max_new_tokens=64)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return {"prompt": input.prompt, "prediction": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
from sagemaker.huggingface import HuggingFaceModel # type: ignore
import sagemaker # type: ignore
import boto3 # type: ignore

# Update this to your AWS region
region = "eu-north-1"
sagemaker_session = sagemaker.Session()
role = "arn:aws:iam::..........:role/Sagemaker_S3_Role"

# Model location on S3
model_location = "s3://............./model.tar.gz"

# Environment variables
env = {
    'HF_TASK': 'text2text-generation'
}

# Define the model
huggingface_model = HuggingFaceModel(
    model_data=model_location,
    role=role,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",
    env=env,
    sagemaker_session=sagemaker_session
)

# Deploy model
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name="huggingface-pytorch-inference-2025-04-21-16-09-01-008"
)

print("Deployed to endpoint:", predictor.endpoint_name)

In [ ]:
from fastapi import FastAPI, HTTPException # type: ignore
from pydantic import BaseModel # type: ignore
import boto3 # type: ignore
import json
import os

# AWS setup
REGION = "eu-north-1"
ENDPOINT_NAME = "huggingface-pytorch-inference-..........."

runtime = boto3.client("sagemaker-runtime", region_name=REGION)

# FastAPI setup
app = FastAPI(title="SageMaker LLM Inference API")

class InputText(BaseModel):
    prompt: str

@app.post("/predict/")
def predict(input: InputText):
    try:
        payload = {
            "inputs": input.prompt
        }

        response = runtime.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            ContentType="application/json",
            Body=json.dumps(payload)
        )

        result = json.loads(response["Body"].read().decode("utf-8"))

        return {
            "prompt": input.prompt,
            "prediction": result[0]['generated_text'] if isinstance(result, list) else result
        }

    except Exception as e:
        print("🔥 Error occurred while calling SageMaker:", e)  # <--- add this
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
# curl -X POST http://127.0.0.1:8000/predict/ \
#     -H "Content-Type: application/json" \
#     -d '{"prompt": "Classify the question type: How can I find a list of celebrities’ real names?"}'